In [1]:
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm 

import warnings 
warnings.filterwarnings('ignore')

In [2]:
def ud(data,n1):
    data["ud"] = np.sign(data["close"]-data['close'].shift(n1))
    
    ud_cumm=list(np.zeros(n1))
    start = None
    cumm = 0
    for i in range(n1,len(data["date"])):
        if start is None:
            ud_cumm.append(data["ud"][i])
            start = data["ud"][i]
            cumm = data["ud"][i]
        else:
            if data["ud"][i] == start:
                cumm += data["ud"][i]
                ud_cumm.append(cumm)
            else:
                ud_cumm.append(data["ud"][i])
                start = data["ud"][i]
                cumm = data["ud"][i]
    data["ud_cumm"] = ud_cumm
    return data

In [3]:
def count(data,n2,n3):
    
    data["start_or_not"]=None
    data["sell_count"] = None
    data["buy_count"] = None
    data["signal"]=None
    data["price_high"] =None
    data["price_low"] =None
    count_buy = 0
    count_sell = 0
    price_low = None
    price_high = None
    buying_begin=False
    selling_begin = False
    last_buy_price=None
    last_sell_price =None
    
    for i in range(0,len(data["date"])):            
        if buying_begin and count_buy<n3: #买入计数
            price_low = min(data["low"][i],price_low) if price_low else data["low"][i]
            if data["close"][i]>=data["high"][i-2] and data["high"][i]>data["high"][i-1]:
                if count_buy == 0:
                    count_buy += 1
                    data.loc[i,("buy_count")] = count_buy
                    last_buy_price = data["close"][i]
                elif count_buy != 0 and data["close"][i]>last_buy_price:
                    count_buy += 1
                    data.loc[i,("buy_count")] = count_buy
                    last_buy_price = data["close"][i]  
            
                    
        if selling_begin and count_sell<n3: #卖出计数
            price_high = max(data["high"][i],price_high) if price_high else data["high"][i]
            if data["close"][i]<=data["low"][i-2] and data["low"][i]<data["low"][i-1]:
                if count_sell == 0:
                    count_sell += 1
                    data.loc[i,("sell_count")] = count_sell
                    last_sell_price = data["close"][i]
                elif count_sell!=0 and data["close"][i] < last_sell_price:
                    count_sell += 1
                    data.loc[i,("sell_count")] = count_sell
                    last_sell_price = data["close"][i]
            
                    
        if data["ud_cumm"][i] == n2: #卖出启动
            data.loc[i,("start_or_not")] =-1
            count_sell = 0 #初始化计数次数
            data.loc[i,("sell_count")] = count_sell
            selling_begin = True
            price_high = None
        elif data["ud_cumm"][i] == -n2: #买入启动
            data.loc[i,("start_or_not")] =1
            count_buy = 0 #初始化计数次数
            data.loc[i,("buy_count")] = count_buy
            buying_begin = True
            price_low = None
                    
        if count_buy == n3:
            data["signal"][i] = 1
            count_buy = 0
            data.loc[i,("price_low")] = price_low
            buying_begin = False
            
        if count_sell == n3:
            data["signal"][i] = -1
            count_sell = 0
            data.loc[i,("price_high")] = price_high
            selling_begin = False

    return data

In [4]:
def signal(data):
    
    day_first_idexs = data.drop_duplicates('date').index.values #取出相同date的第一个日期index 如0 240 480
    day_idexs = np.hstack([day_first_idexs,data.shape[0]]) #加上最后一个index 如0 240 480。。。57840（方便后面取每日区间）
    my_data_dfs = []
    for day_begin, day_end in zip(day_idexs[:-1], day_idexs[1:]):#取每一天的区间
        tmp_df = data.iloc[day_begin: day_end, :]    
        tmp_df["new_signal"]=None
        tmp_df["new_signal"][tmp_df["time"]< '14:40:00'] = tmp_df.signal.shift(1)
        tmp_df["stop or not"] =False
        direction = None
        init = False
        count = 0
        
        for i in range(day_begin, day_end):
            
            if tmp_df["time"][i] == '14:40:00':
                tmp_df.loc[i,("new_signal")] = -count
#                 print("------------------------------------")
            elif tmp_df["time"][i] < '14:40:00':
                if direction ==None and tmp_df["signal"][i] ==1:
                    direction =1
                    price_low = data["price_low"][i]
                    count+=1
#                     print("first")

                elif direction ==None and tmp_df["signal"][i] ==-1 :
                    direction = -1
                    count+=-1
#                     print("second")

                elif direction == 1:     
#                     print(tmp_df["low"][i], price_low)
                    if tmp_df["low"][i] - price_low <= 0 and tmp_df["time"][i] < '14:39:00':
                        tmp_df.loc[i+1,("new_signal")] = -1
                        count-=1
                        tmp_df.loc[i+1,("price_low")] = price_low
                        tmp_df.loc[i+1,("stop or not")] = True
                        direction = -1
#                         print("third")
                    elif tmp_df["signal"][i] ==1:
                        price_low = data["price_low"][i]
                        count+=1
#                         print("fourth")
                    elif tmp_df["signal"][i] ==-1:
                        count-=1
#                         print("fourth2")
                elif direction == -1: 
                    if tmp_df["signal"][i] ==1:
                        price_low = data["price_low"][i]
                        count+=1
                        direction =1
#                         print("fifth")
                    elif tmp_df["signal"][i] ==-1:
                        count-=1
                        direction = -1
#                         print("six")

            
        
        '''
            signal：上述策略中的信号，有1，-1，0 三种取值
            new_signal 针对只能开平多仓的新信号点
            my_signal 在new_signal基础上加上日内平仓 表现为14:40:00 发出-1/1信号
            init是初始化阶段，即第一次进行开仓或者平仓操作，初始化阶段后将init设置为false
            
            
            特殊：由于日内平仓，当时间>14:40:00，就算有signal信号，也不进行开仓操作。
        '''

        my_data_dfs.append(tmp_df)
    """ 拼接得到新DataFrane """
    my_data = pd.concat(my_data_dfs)
    return my_data           

In [5]:
#持仓计算
def position(data, shift_period=1):
    signal = data['new_signal']
    position = signal.fillna(method='ffill').shift(shift_period).fillna(0)
    data['position'] = position
    return data

## 算实际开仓

In [10]:
#读取数据
my_data=pd.read_csv(
    '510300_2021.csv', sep=',', parse_dates=True)
my_data_1 = my_data[my_data["date"]>='2021-01-01']
my_data_1 = my_data_1[my_data_1["dates"]<='2021-05-31 15:00:00']
my_data_1 =my_data_1.reset_index(drop=True)
my_data_1

,date,time,dates,open,high,low,close
0,2021-01-04,09:31:00,2021-01-04 09:31:00,5.280,5.298,5.278,5.288
1,2021-01-04,09:32:00,2021-01-04 09:32:00,5.290,5.290,5.283,5.283
2,2021-01-04,09:33:00,2021-01-04 09:33:00,5.283,5.286,5.281,5.283
3,2021-01-04,09:34:00,2021-01-04 09:34:00,5.281,5.281,5.270,5.272
4,2021-01-04,09:35:00,2021-01-04 09:35:00,5.274,5.274,5.263,5.265
...,...,...,...,...,...,...,...
23275,2021-05-31,14:56:00,2021-05-31 14:56:00,5.335,5.338,5.335,5.338
23276,2021-05-31,14:57:00,2021-05-31 14:57:00,5.338,5.339,5.337,5.338
23277,2021-05-31,14:58:00,2021-05-31 14:58:00,5.338,5.338,5.337,5.338
23278,2021-05-31,14:59:00,2021-05-31 14:59:00,5.338,5.339,5.336,5.338


## 实际开仓（加上6s的频）

In [11]:
my_data_3s=pd.read_csv(
    '510300_2021_6s_1.csv', sep=',', parse_dates=True)
my_data_3s_1 = my_data_3s[my_data_3s["date"]>='2021-01-01']
my_data_3s_1 = my_data_3s_1[my_data_3s_1["dates"]<='2021-05-31 15:00:00']
my_data_3s_1 = my_data_3s_1[my_data_3s_1["time"]>='09:31:00']
my_data_9s_1 =my_data_3s_1.reset_index(drop=True)
my_data_9s_1

,date,time,dates,open,high,low,close,buy1,bc1,buy2,bc2,sale1,sc1,sale2,sc2
0,2021-01-04,09:31:00,2021-01-04 09:31:00,5.289,5.289,5.288,5.288,5.288,479000,5.287,77200,5.289,499300,5.290,126800
1,2021-01-04,09:31:06,2021-01-04 09:31:06,5.290,5.290,5.290,5.290,5.289,2000,5.287,477400,5.290,54600,5.291,300800
2,2021-01-04,09:31:12,2021-01-04 09:31:12,5.288,5.290,5.288,5.290,5.290,971900,5.289,140400,5.291,1200,5.292,14400
3,2021-01-04,09:31:18,2021-01-04 09:31:18,5.290,5.290,5.290,5.290,5.289,456800,5.288,4300,5.290,568800,5.291,11700
4,2021-01-04,09:31:24,2021-01-04 09:31:24,5.290,5.290,5.290,5.290,5.289,65500,5.288,609800,5.290,59400,5.291,156700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231922,2021-05-31,14:59:36,2021-05-31 14:59:36,5.338,5.338,5.338,5.338,5.338,175300,5.337,322483,5.339,676700,5.340,1861600
231923,2021-05-31,14:59:42,2021-05-31 14:59:42,5.338,5.339,5.338,5.339,5.338,169900,5.337,317083,5.339,708400,5.340,1559600
231924,2021-05-31,14:59:48,2021-05-31 14:59:48,5.339,5.339,5.339,5.339,5.338,165200,5.337,307583,5.339,1871600,5.340,965600
231925,2021-05-31,14:59:54,2021-05-31 14:59:54,5.339,5.339,5.338,5.338,5.338,253500,5.337,304283,5.339,1633800,5.340,965600


In [12]:
#计算新的开平仓收益
def new_earning(data,init_money,init_volume,r0):
    
    '''
        r0:交易单边手续费
        times:每日交易次数  = 固定为3次
        初始化持有金额和仓位（初始持有：两者之和）：
            init_money              初始持有金额
            init_volume             初始持有仓位 
        
        描述变量：
            hold_volume             持有仓位
            all_volume              数量上等于hold_volume 为方便计算引入的变量，目的：比如多仓下平仓，用hold_volume来计算，
                                    判断平仓点为hold_volume=0,而all_volume保留为平仓前的多仓持有仓位
                                    
            yesterday_volume        昨日持仓量
            today_volume            今日持仓量
            change_volume           单次买卖操作变化持仓量

            hold_money              持有金额,初始化为初始持有金额
            all_money               数量上等于hold_money 为方便计算引入的变量
            change_money            单次买卖操作变化金额
            everyday_money          list 每日初始持有金额，第一天为init_money
            
            change_time             单次买卖操作时间（second）
            
            new_price               平仓开仓的价格，all_money/all_volume
            per_time_volume         每次操作的量
            
        标识：     
            buy_signal              买入标识
            sell_signal             卖出标识
            begin_time              每次操作时开仓时间（用来计算每次买卖的所用时长）
            
    '''
    data["yesterday_volume"]=None
    data["today_volume"]=None
    data["change_volume"]=None
    data["all_money"]=None
    data["change_money"]=None
    data["new_position"]=None
    data["new_price"]=None
    data["change_time"]=None
    hold_money=init_money                   #初始持有金额
    all_money=hold_money
    yesterday_volume = init_volume           #昨日及之前持仓
    today_volume = 0                      #今日持仓
    everyday_money=[]                     #每日初始金额
    times =3
    per_time_volume =init_volume/times   #计算每次操作的量
#     print(times,(data[data['new_signal']>0].shape[0]+data[data['new_signal']<0].shape[0]))
    
    day_first_idexs = data.drop_duplicates('date').index.values #取出相同date的第一个日期index
    day_idexs = np.hstack([day_first_idexs,data.shape[0]]) #加上最后一个index （方便后面取每日区间）
    my_data_dfs = []
    for day_begin, day_end in zip(day_idexs[:-1], day_idexs[1:]):#取每一天的区间
        tmp_df = data.iloc[day_begin: day_end, :]   
        #yesterday_volume                  #每日初始持有仓位
        yesterday_volume += today_volume   #每日初始持有仓位 = 昨日的留存的yesterday_volume + 昨日的today_volume
        everyday_money.append(all_money)   #每日初始持有金额计入

        
        #初始化今日量，买卖signal，开仓时间
        today_volume =0                    
        buy_signal = 0
        sell_signal = 0
        begin_time = 0
        
        for i in range(day_begin, day_end):
            if(tmp_df["time"][i]<'14:40:00'): 
                '''
                    在14:40:00前: 
                    买入条件：发出买入信号（new_signal=1），同时今日买入的仓位<初始仓位
                        初始化：发出buy_signal,hold_volume单次初始持仓=0，持有金额=all_money
                                单次所需买入的量为min(per_time_volume，init_volume-today_volume)
                                begin_time = 发出信号的时间
                        
                    卖出条件：发出卖出信号（new_signal=-1），同时昨日持仓>0
                        初始化：发出sell_signal,hold_volume单次卖出的量min(yesterday_volume,per_time_volume)，
                                per_money 单次操作金额=0
                                begin_time = 发出信号的时间
                '''                
                
                if(tmp_df["new_signal"][i]==1 and today_volume<init_volume):           
                    buy_signal=1
                    hold_volume=0
                    hold_money=all_money
                    need_volume=min(per_time_volume,init_volume-today_volume)
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("today_volume")]=today_volume
                    begin_time = tmp_df["time"][i]


                elif(tmp_df["new_signal"][i]==-1):        
                    sell_signal=1
                    hold_volume =  min(yesterday_volume,per_time_volume)
                    all_volume = hold_volume
                    per_money = 0
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("today_volume")]=today_volume  
                    begin_time = tmp_df["time"][i]
                    
            
            elif(tmp_df["time"][i]=='14:40:00' ):
                '''
                    在14:40:00后: 日内平仓操作
                    买入条件：today_volume+yesterday_volume-init_volume<0
                        初始化：发出buy_signal,hold_volume单次初始持仓=0，持有金额=all_money
                                单次所需买入的量为 init_volume-(today_volume+yesterday_volume)
                                begin_time = 发出信号的时间
                        
                    卖出条件：today_volume+yesterday_volume-init_volume>0
                        初始化：发出sell_signal,hold_volume单次卖出的量，
                                per_money 单次操作金额=0
                                begin_time = 发出信号的时间
                '''
#                 if (buy_signal==1):
#                     hold_money=all_money
#                     buy_signal=0
#                 elif(sell_signal==1):
#                     sell_signal=0
                    
                if(today_volume+yesterday_volume-init_volume>0):  
                    if(yesterday_volume>=today_volume+yesterday_volume-init_volume):
                        hold_volume =  today_volume+yesterday_volume-init_volume
                    else:
                        hold_volume =  yesterday_volume
                    sell_signal=1
                    all_volume = hold_volume
                    per_money = 0
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("today_volume")]=today_volume
                    begin_time = tmp_df["time"][i]
                elif(today_volume+yesterday_volume-init_volume<0): 
                    buy_signal=1
                    hold_volume=0
                    hold_money=all_money
                    need_volume = init_volume-(today_volume+yesterday_volume)
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("today_volume")]=today_volume
                    begin_time = tmp_df["time"][i]
                     
                
            if(buy_signal ==1):
            
                '''
                    对于每次开仓过程中有三种情况：
                    1.持有金额>卖1价*卖1量+卖2价*卖2量 
                    2.卖1价*卖1量+卖2价*卖2量>=持有金额>卖2价*卖2量 
                    3.持有金额<=卖1价*卖1量  此时以卖1价买入
                    hold_volume 本次买入的仓位 0-->
                    today_volume 今日仓位
                    hold_money 持有的钱(变量)
                    all_money 本次操作之前持有的钱 --> hold_money
                '''
                #计算两档金额
                money_1 = tmp_df["sale1"][i]*tmp_df["sc1"][i]
                money_2 = tmp_df["sale2"][i]*tmp_df["sc2"][i]

                if(need_volume-hold_volume>tmp_df["sc1"][i] and need_volume-hold_volume<=tmp_df["sc1"][i]+tmp_df["sc2"][i]):
                    #需要再扫一档 +二档的一部分
                    need_money = (money_1 + (need_volume-hold_volume-tmp_df["sc1"][i])*tmp_df["sale2"][i])*(1+r0)
                    if(hold_money>=need_money):
                        #钱够扫
                        hold_volume = need_volume
                        hold_money -= need_money
                    else:
                        #钱只够扫一档+二档的小部分， 不能扫满所需二档
                        if(hold_money>money_1*(1+r0)):
                            hold_volume = hold_volume + tmp_df["sc1"][i] + (hold_money - money_1*(1+r0))*(1-r0)/tmp_df["sale2"][i]
                            hold_money = 0
                        else:
                            #钱只够扫一档一部分
                            hold_volume = hold_volume + hold_money*(1-r0)/tmp_df["sale1"][i]
                            hold_money = 0  

                elif(need_volume-hold_volume>tmp_df["sc1"][i]+tmp_df["sc2"][i]):   
                    #需要再扫一档 +二档
                    need_money = (money_1 + money_2)*(1+r0)
                    if(hold_money>need_money):
                        #钱够扫
                        hold_volume += tmp_df["sc1"][i]+tmp_df["sc2"][i]
                        hold_money -= need_money
                    else:
                        #钱只够扫一档+二档的小部分， 不能扫满所需二档
                        if(hold_money>money_1*(1+r0)):
                            hold_volume = hold_volume + tmp_df["sc1"][i] + (hold_money - money_1*(1+r0))*(1-r0)/tmp_df["sale2"][i]
                            hold_money = 0
                        else:
                            #钱只够扫一档一部分
                            hold_volume = hold_volume + hold_money*(1-r0)/tmp_df["sale1"][i]
                            hold_money = 0  

                else:
                    #需要再扫一档 
                    need_money = (need_volume-hold_volume)*tmp_df["sale1"][i]*(1+r0)
                    if(hold_money>need_money):
                        #钱够扫
                        hold_volume = need_volume
                        hold_money -= need_money
                    else:
                        #钱只够扫一档一部分
                        hold_volume = hold_volume + hold_money*(1-r0)/tmp_df["sale1"][i]
                        hold_money = 0  

                #如果上次买入操作在14:39:40前还没结束，那么结束此次买入操作，等到14:40:00进行日内平仓
                if(hold_volume>=need_volume or hold_money== 0 or (tmp_df.loc[i,("time")]>'14:39:40' and tmp_df.loc[i,("time")]<'14:40:00')):
                    today_volume+=hold_volume
                    tmp_df.loc[i,("today_volume")]=today_volume
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("change_volume")]=hold_volume
                    tmp_df.loc[i,("change_money")]=all_money-hold_money
                    tmp_df.loc[i,("new_price")]=(all_money-hold_money)/(hold_volume)      
                    all_money = hold_money
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("change_time")]=(datetime.datetime.strptime(tmp_df.loc[i,("time")], "%H:%M:%S")-datetime.datetime.strptime(begin_time, "%H:%M:%S")).seconds
                    buy_signal=0
                    
                
                           
            elif(sell_signal ==1):
                '''
                    对于每次平仓过程中有三种情况：
                    1.持有仓位>买1量+买2量 
                    2.买1量+买2量>=持有仓位>买1量 
                    3.持有仓位<=买1量  此时以买1价都卖出
                    per_money 卖出的钱
                    hold_money/all_money 总持有钱
                    hold_volume 初始为per_time_volume 每次卖出的量 --> 0
                    all_volume 初始的hold_volume --> 0
                '''
                money_1 = tmp_df["buy1"][i]*tmp_df["bc1"][i]
                money_2 = tmp_df["buy2"][i]*tmp_df["bc2"][i]
                
                #如果上次卖出操作在14:39:40前还没结束，那么结束此次卖出操作，等到14:40:00进行日内平仓
                if(hold_volume-tmp_df["bc1"][i]-tmp_df["bc2"][i]>0 and (tmp_df.loc[i,("time")]<='14:39:40' or tmp_df.loc[i,("time")]>='14:40:00')):
                    hold_volume=hold_volume - tmp_df["bc1"][i]-tmp_df["bc2"][i]
                    per_money = per_money+(money_1+money_2)*(1-r0)
                else: 
                    if(tmp_df.loc[i,("time")]<='14:39:40' or tmp_df.loc[i,("time")]>='14:40:00'):
                        if (hold_volume-tmp_df["bc1"][i]-tmp_df["bc2"][i]<=0 and hold_volume-tmp_df["bc1"][i]>0):
                            per_money += (money_1+(hold_volume-tmp_df["bc1"][i])*tmp_df['buy2'][i])*(1-r0)
                        else:
                            per_money +=  hold_volume*tmp_df["buy1"][i]*(1-r0) 
                        # 仓位在卖掉后为0
                        hold_volume=0
                    elif(tmp_df.loc[i,("time")]>'14:39:40'and tmp_df.loc[i,("time")]<'14:40:00'):
                        all_volume = all_volume-hold_volume
                        
                    yesterday_volume -= all_volume
                    tmp_df.loc[i,("yesterday_volume")]=yesterday_volume
                    tmp_df.loc[i,("today_volume")]=today_volume
                    tmp_df.loc[i,("change_volume")]=all_volume
                    tmp_df.loc[i,("change_money")]=per_money
                    hold_money += per_money
                    all_money=hold_money
                    tmp_df.loc[i,("all_money")]=all_money
                    tmp_df.loc[i,("new_price")]=(per_money)/all_volume
                    tmp_df.loc[i,("change_time")]=(datetime.datetime.strptime(data.loc[i,("time")], "%H:%M:%S")-datetime.datetime.strptime(begin_time, "%H:%M:%S")).seconds
                    all_volume=0
                    sell_signal=0
                
            
        my_data_dfs.append(tmp_df)
    """ 拼接得到新DataFrane """
    my_data = pd.concat(my_data_dfs) 
    return my_data,everyday_money

In [13]:
#算每日操作次数 每日操作金额 每日操作平均所需时长
def per_day(data,money,everyday_money):
    '''
       每日操作次数 per_day_count 
       每日操作量 per_day_change_volume
       每日操作金额 per_day_change_money
       每次操作平均所需时长 per_day_change_time
       最大回撤 drawback
    
    '''
#     count_num = data[(data['new_signal']>0) or (data['new_signal']<0)].shape[0] #操作次数
    count_num = data[data['new_signal']>0].shape[0]+data[data['new_signal']<0].shape[0] #操作次数
    total_change_volume = data['change_volume'].dropna().values.sum()
    total_change_money = data['change_money'].dropna().values.sum()
    total_change_time = data['change_time'].dropna().values.sum()
    total_days = data.drop_duplicates('date').index.values.size
    per_day_count = 3
    per_day_change_volume =total_change_volume /total_days
    per_day_change_money = total_change_money /total_days
    per_count_change_time = total_change_time /count_num
    earning = (my_new_situation_earning['all_money'].dropna().values[-1]/money -1)*100
    print(my_new_situation_earning['all_money'].dropna().values[-1],money)
    
    drawback_list = []
    for i in range(0,len(everyday_money)):
        drawback_list.append((everyday_money[i]-min(everyday_money[i:]))/everyday_money[i])
    drawback = max(drawback_list)*100
    return per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning,drawback

# per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning = per_day(my_new_situation_earning,20000000)
# per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning

In [ ]:
performance=[]
drawback=[]
per_money_day=[]
N1 = np.arange(1,11)#11 6 7
N2 = np.arange(1,11)
N3 = np.arange(5,8)
money=[10000000,20000000]
df_par = pd.DataFrame()
df_par["n1"]=np.zeros(600)
df_par["n2"]=np.zeros(600)
df_par["n3"]=np.zeros(600)
df_par["per_day_count"]=np.zeros(600)
df_par["per_day_change_volume"]=np.zeros(600)
df_par["per_day_change_money"]=np.zeros(600)
df_par["per_count_change_time"]=np.zeros(600)
df_par["performance"]=np.zeros(600)
df_par["drawback"]=np.zeros(600)
# print(N1,N2,N3)
m1,m2,m3,m4,m5,m6,m7,m8,m9,m10 =[],[],[],[],[],[],[],[],[],[]
for n1 in N1:
    for n2 in N2:
        for n3 in N3:
            my_data_ud = ud(my_data_1 , n1)
            my_data_count = count(my_data_ud,n2,n3)
            my_data_signal = signal(my_data_count)
            my_new_situation = pd.merge(my_data_9s_1,my_data_signal,on='dates',how='left')
            my_new_situation =my_new_situation.drop(my_new_situation.columns[[15,16,17,18,19,20,21,22,23,24,25,26,27,28,30]], axis=1)
            my_new_situation.rename(columns={'date_x':'date','time_x':'time','open_x':'open','high_x':'high','low_x':'low','close_x':'close'},inplace=True)
                        
            for n4 in money:
                my_new_situation_earning,everyday_money=new_earning(my_new_situation,n4,5000000,0.00005)
                per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning,drawback = per_day(my_new_situation_earning,n4,everyday_money)
                m1.append(int(n1))
                m2.append(int(n2))
                m3.append(int(n3))
                m4.append(int(n4))
                m5.append(per_day_count)
                m6.append(per_day_change_volume)
                m7.append(per_day_change_money)
                m8.append(per_count_change_time)
                m9.append(earning)
                m10.append(drawback)
                print(n1,n2,n3,n4,earning)

df_par["n1"]=m1
df_par["n2"]=m2
df_par["n3"]=m3
df_par["money"]=m4
df_par["per_day_count"]=m5
df_par["per_day_change_volume"]=m6
df_par["per_day_change_money"]=m7
df_par["per_count_change_time"]=m8
df_par["performance"]=m9
df_par["drawback"]=m10
df_par

9278467.123146929 10000000
1 1 5 10000000 -7.215328768530716
19411804.2987271 20000000
1 1 5 20000000 -2.940978506364511
9610880.808626747 10000000
1 1 6 10000000 -3.891191913732528
19550470.221602377 20000000
1 1 6 20000000 -2.247648891988119
10007398.567363646 10000000
1 1 7 10000000 0.07398567363645636
20007466.146482803 20000000
1 1 7 20000000 0.037330732414009304
9298106.01194742 10000000
1 2 5 10000000 -7.01893988052581
19532536.81815339 20000000
1 2 5 20000000 -2.337315909233051
10842349.17833188 10000000
1 2 6 10000000 8.423491783318804
21206333.61539942 20000000
1 2 6 20000000 6.03166807699711
10752439.224243987 10000000
1 2 7 10000000 7.524392242439881
21015824.505413257 20000000
1 2 7 20000000 5.079122527066282
9624334.55792591 10000000
1 3 5 10000000 -3.7566544207409014
19911750.9909197 20000000
1 3 5 20000000 -0.4412450454014949
9433033.669774447 10000000
1 3 6 10000000 -5.669663302255534
19530007.716563135 20000000
1 3 6 20000000 -2.349961417184321
10273682.156163348 1000

In [ ]:
df_par.to_csv("my_result_6s_1_2021.csv")

## 测试

In [18]:
my_data_ud = ud(my_data_1 , 6)
my_data_count = count(my_data_ud,4,5)
my_data_signal = signal(my_data_count)
my_new_situation = pd.merge(my_data_9s_1,my_data_signal,on='dates',how='left')
my_new_situation =my_new_situation.drop(my_new_situation.columns[[15,16,17,18,19,20,21,22,23,24,25,26,27,28,30]], axis=1)
my_new_situation.rename(columns={'date_x':'date','time_x':'time','open_x':'open','high_x':'high','low_x':'low','close_x':'close'},inplace=True)
n4=10000000
my_new_situation_earning,everyday_money=new_earning(my_new_situation,n4,5000000,0.00005)
per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning,drawback = per_day(my_new_situation_earning,n4,everyday_money)
per_day_count,per_day_change_volume,per_day_change_money,per_count_change_time,earning,drawback

13668422.864113312 10000000


(3,
 9659944.164323686,
 50730279.80330402,
 24.88213851761847,
 36.68422864113312,
 7.9446644985152925)

In [19]:
n4=10000000
my_new_situation_earning,everyday_money=new_earning(my_new_situation,n4,5000000,0.00005)


In [20]:
my_new_situation_earning[(my_new_situation_earning["yesterday_volume"]>0)|(my_new_situation_earning["today_volume"]>0)|(my_new_situation_earning["all_money"]>0)][100:150]

,date,time,dates,open,high,low,close,buy1,bc1,buy2,...,sc2,new_signal,yesterday_volume,today_volume,change_volume,all_money,change_money,new_position,new_price,change_time
18887,2021-01-13,14:36:00,2021-01-13 14:36:00,5.616,5.616,5.616,5.616,5.616,135200,5.615,...,108300,-1,0.0,5000000.0,0.0,9940713.89082,0.0,None,NaN,0
19228,2021-01-14,09:41:00,2021-01-14 09:41:00,5.620,5.620,5.618,5.619,5.617,1587600,5.616,...,590700,-1,3333333.333333,0,1666666.666667,19301833.41144,9361119.52062,None,5.616672,0
19508,2021-01-14,10:09:00,2021-01-14 10:09:00,5.577,5.577,5.576,5.576,5.576,226600,5.575,...,72500,-1,3333333.333333,0,None,19301833.41144,None,None,None,None
19509,2021-01-14,10:09:06,2021-01-14 10:09:06,5.577,5.578,5.577,5.578,5.577,994400,5.576,...,676100,NaN,1666666.666667,0,1666666.666667,28594610.14937,9292776.73793,None,5.575666,6
19628,2021-01-14,10:21:00,2021-01-14 10:21:00,5.588,5.588,5.586,5.586,5.586,639400,5.585,...,1084500,1,1666666.666667,0,None,28594610.14937,None,None,None,None
19629,2021-01-14,10:21:06,2021-01-14 10:21:06,5.588,5.589,5.588,5.589,5.588,13700,5.587,...,655900,NaN,1666666.666667,1666666.666667,1666666.666667,19278800.415505,9315809.733865,None,5.589486,6
20248,2021-01-14,11:23:00,2021-01-14 11:23:00,5.603,5.604,5.603,5.604,5.603,2600,5.602,...,2376600,1,1666666.666667,3333333.333333,1666666.666667,9937266.095475,9341534.32003,None,5.604921,0
20328,2021-01-14,13:01:00,2021-01-14 13:01:00,5.585,5.585,5.585,5.585,5.584,77800,5.583,...,1500000,-1,1666666.666667,3333333.333333,None,9937266.095475,None,None,None,None
20330,2021-01-14,13:01:12,2021-01-14 13:01:12,5.584,5.584,5.582,5.582,5.581,162100,5.580,...,600000,NaN,0.0,3333333.333333,1666666.666667,19239621.65444,9302355.558965,None,5.581413,12
20588,2021-01-14,13:27:00,2021-01-14 13:27:00,5.572,5.572,5.572,5.572,5.571,114300,5.570,...,1017000,-1,0.0,3333333.333333,0.0,19239621.65444,0.0,None,NaN,0


In [31]:
my_new_situation_earning[31668:31693]

,date,time,dates,open,high,low,close,buy1,bc1,buy2,...,sc2,new_signal,yesterday_volume,today_volume,change_volume,all_money,change_money,new_position,new_price,change_time
31668,2022-01-28,14:40:27,2022-01-28 14:40:27,4.576,4.576,4.576,4.576,4.576,86300,4.573,...,772700,NaN,1e+06,4e+06,3e+06,1.0167e+07,1.37331e+07,None,4.57769,27
31669,2022-01-28,14:40:36,2022-01-28 14:40:36,4.573,4.573,4.573,4.573,4.572,18000,4.571,...,301600,NaN,None,None,None,None,None,None,None,None
31670,2022-01-28,14:40:45,2022-01-28 14:40:45,4.572,4.572,4.572,4.572,4.571,1200,4.570,...,287700,NaN,None,None,None,None,None,None,None,None
31671,2022-01-28,14:40:54,2022-01-28 14:40:54,4.572,4.572,4.572,4.572,4.570,94500,4.569,...,232300,NaN,None,None,None,None,None,None,None,None
31672,2022-01-28,14:41:00,2022-01-28 14:41:00,4.571,4.571,4.571,4.571,4.568,1069600,4.567,...,893000,None,0,4e+06,1e+06,2.75764e+07,1.74094e+07,None,17.4094,60
31673,2022-01-28,14:41:09,2022-01-28 14:41:09,4.570,4.570,4.570,4.570,4.569,7900,4.568,...,7900,NaN,None,None,None,None,None,None,None,None
31674,2022-01-28,14:41:18,2022-01-28 14:41:18,4.570,4.570,4.570,4.570,4.569,13500,4.568,...,202000,NaN,None,None,None,None,None,None,None,None
31675,2022-01-28,14:41:27,2022-01-28 14:41:27,4.569,4.569,4.569,4.569,4.568,527900,4.567,...,280600,NaN,None,None,None,None,None,None,None,None
31676,2022-01-28,14:41:36,2022-01-28 14:41:36,4.570,4.570,4.570,4.570,4.569,300,4.568,...,540600,NaN,None,None,None,None,None,None,None,None
31677,2022-01-28,14:41:45,2022-01-28 14:41:45,4.568,4.568,4.568,4.568,4.567,300700,4.566,...,5400,NaN,None,None,None,None,None,None,None,None
